In [ ]:
# Autores: 
# Junco de las Heras Valenzuela junco.heras@estudiante.uam.es
# Marta Vaquerizo Núñez marta.vaquerizo@estudiante.uam.es
# Grupo 2301, Doble Grado.

## 4. Predicción de fugas de clientes en una compañía telefónica (7 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotaación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [1]:
import pandas as pd

In [2]:
datos_construccion = pd.read_csv("./fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./fuga_clientes_empresa_telefonica_explotacion.csv")

In [3]:
datos_construccion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [4]:
datos_explotacion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [5]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:

sum(datos_construccion.columns != datos_explotacion.columns)

0

In [6]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [7]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

atributos = list(datos_construccion.columns)
# Se elimina el atributo que vamos a predecir
atributos.remove("Churn Status")
# Customer ID es irrelevante en nuestro caso ya que todos
# los clientes tienen un ID distinto, no debería influir en nuestro modelo.
atributos.remove("Customer ID")
# Como tenemos network_age, el tiempo en dias,
# no necesitamos el tiempo en meses, que da menos precision.
atributos.remove("Customer tenure in month")

# Se elimina las instancias con los network_age menores que cero (los hay).
mask = datos_construccion['network_age'] >= 0
df = datos_construccion[mask]

X = df[atributos].values
y = df["Churn Status"].values

# Comprobamos los 5 modelos de clasificadores que tenemos
# para elegir cual vamos a usar.

# 57 score.
hls = [30 for _ in range(1, 8)]
clf = MLPClassifier(hidden_layer_sizes=hls, max_iter=1000,alpha=0)

scores = cross_val_score(clf, X, y, cv=5)
print('MLP scores:', scores.mean(), '+/-', scores.std())

# 76 score.
clf = DecisionTreeClassifier(max_depth=5)

scores = cross_val_score(clf, X, y, cv=5)
print('DecisionTree scores:', scores.mean(), '+/-', scores.std())

# 63 score.
clf = KNeighborsClassifier(n_neighbors=14)

scores = cross_val_score(clf, X, y, cv=5)
print('KNN scores:', scores.mean(), '+/-', scores.std())

# 62 score.
clf = GaussianNB()

scores = cross_val_score(clf, X, y, cv=5)
print('Naive Bayes scores:', scores.mean(), '+/-', scores.std())

# 62 score.
clf = LogisticRegression(C=1e10,solver='lbfgs', max_iter=1000)

scores = cross_val_score(clf, X, y, cv=5)
print('Logistic Regression scores:', scores.mean(), '+/-', scores.std())


MLP scores: 0.5487464840406017 +/- 0.03351998038829762
DecisionTree scores: 0.7679629856100443 +/- 0.06198384808960889
KNN scores: 0.6325791855203621 +/- 0.07514878451860649
Naive Bayes scores: 0.6272430801842567 +/- 0.019958048609438228
Logistic Regression scores: 0.6200766377236965 +/- 0.031822096525812325


In [9]:
# El clasificador que da mejor score es el de Decision Tree, lo usamos.
clf = clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X, y)

atributos2 = list(datos_explotacion.columns)
atributos2.remove("Churn Status")
atributos2.remove("Customer ID")
atributos2.remove("Customer tenure in month")

# Elimina instancias con elementos negativos de network_age.
# Se puede comprobar con df.describe()
mask2 = datos_explotacion['network_age'] >= 0
df2 = datos_explotacion[mask2]


nuevoSet = df2[atributos2].values
# Devuelve una lista con elementos de la forma [prob0 prob1]
# donde prob0 es la probabilidad de que la instancia sea de la 
# clase 0 y prob1 la probabilidad de que la instancia sea de la
# clase 1.
probabilidades = clf.predict_proba(nuevoSet)
# Resultado tiene los elementos [prob1, cont] donde prob1 es la probabilidad
# de que el cliente se vaya de la compañia y cont es el numero de 
# cliente en el dataset, para luego sacar el ID. 
resultado = []
cont = 0
for el in probabilidades:
    resultado += [[el[1], cont]]
    cont += 1
# Una vez sacado resultado se ordena de mayor a menor prob1.
resultado.sort(reverse = True)
ids = []
cont = 0
# Se cogen los 100 primeros resultads (clientes con
# mayor probabilidad de irse) y se guardan en ids.
for el in resultado:
    if cont >= 100:
        break
    ids += [df2['Customer ID'][el[1]]]
    cont += 1


In [10]:
# Se guardan los ids en un fichero csv nuevo.
my_df = pd.DataFrame(ids) 
my_df.to_csv('fichero_predicciones.csv', index=False, header=['Customer ID'], sep='\n')